## 一般形式的条件随机场，可以展开成非归一化的得分矩$[M]$阵形式

$
\begin{align}
P(y|x) &= \frac{1}{Z}\exp(score(y,x))\\
     &=\frac{1}{Z}\exp(\sum_{l}\sum_{i}w_{l}f_{l}(y_{i+1},y_{i},x)+\sum_{k}\sum_{i}w_{k}f_{k}(y_{i+1},x))\\
     &=\frac{exp(\sum_{s}w_{s}f_{s}(y,x)}{\sum_{y}\exp(\sum_{s}w_{s}f_{s}(y,x)}\\
     &= \prod_{i}\frac{\exp(\sum_{s}f_{s}(y_{i+1},y_{i},x))}{Z}\\
     & =\prod_{i}^{n}[M_{i}]\\
     M_{i} &= \exp(\sum_{s}f_{s}(y_{i+1},y_{i},x))\\
f_{s}(y,x) &=\sum_{i}f_{s}(y_{i+1},y_{i},x)
\end{align}
$


## Bi-LSTM-CRF
会训练两种得分，首先是去打分$P(y_{i}|x)$,这个得分是concat由Bi-LSTM得到的双向feature$ [h_{forward},h_{backward}]$并经过线性映射$W_{2\times{h\_size},tag\_size}$得到对应每个tag的得分；第二个得分是$P(y_{i}|y_{i-1})$，这个得分由转移矩阵$T_{m\times{m}}$刻画，这个转移矩阵是可以通过后向传播训练的。根据论文[Bidirectional LSTM-CRF Models for Sequence Tagging](https://arxiv.org/pdf/1508.01991.pdf)，CRF层具有如下形式$Score(X,Y) = \sum_{t=1}^{T}(A_{i_{t-1},i_{t}}+f_{\theta}(i_{t}))$。整个模型$\mathop{\arg\max}_{\theta}P(y|x)$等价于$\mathop{\arg\min}_{\theta}-\log(P(y|x))$,即$\mathop{\arg\min}_{\theta}\log{\sum_{y}Score(x,y)}-Score(x,y)$,目标路径的得分$Score(x,y)$很容易求得，难点在于如何高效的得到$\sum_{y}Score(x,y)$，使用穷举法得复杂度为$O(Tm^m)$,根据pytorch的实现，使用前向传播算法可以很容易求得所有可能路径的总的得分。前向计算首先定义一个前向变量，然后前向逐步log_sum_exp得到总的logsumexp,具体的细节可以看[最通俗易懂的BiLSTM-CRF模型中的CRF层介绍](https://zhuanlan.zhihu.com/p/44042528)

In [ ]:
# def _forward_alg(self, feats):
#     # calculate in log domain
#     # feats is len(sentence) * tagset_size
#     # initialize alpha with a Tensor with values all equal to -10000.,crf的前向概率初始设置试，为exp（other）=0，exp（start）=1，所以初始值有-1000，0
#     init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
#     init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
#     forward_var = autograd.Variable(init_alphas) # 前向变量
#     if self.use_gpu:
#         forward_var = forward_var.cuda()
#     for feat in feats:
#         emit_score = feat.view(-1, 1)  #feat升维
#         tag_var = forward_var + self.transitions + emit_score # 这实质上是矩阵的加法造作，初次看代码这里是最难理解的，实际操作后很好理解
#         max_tag_var, _ = torch.max(tag_var, dim=1)
#         tag_var = tag_var - max_tag_var.view(-1, 1)
#         forward_var = max_tag_var + torch.log(torch.sum(torch.exp(tag_var), dim=1)).view(1, -1) # ).view(1, -1)  # 更新前向变量
#     terminal_var = (forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]).view(1, -1)  # 得到全路径的前向变量
#     alpha = log_sum_exp(terminal_var)
#     # Z(x)
#     return alpha

实际上前向变量的更新只需要$Forward\_Variable_{t,i}=\log{\sum_{j}\exp(tag\_var_{i,j}^{t})}$  
Pytorch实现转换了这种计算  
$
\begin{align}
\log\sum_{j}\exp(x_{j}) &= \log\sum_{j}\exp(x_{j}-x_{max})\exp(x_{max})\\
                &=\log\exp(x_{max})\sum_{j}\exp(x_{j}-x_{max})\\
                &= x_{max} + \log\sum_{j}\exp(x_{j}-x_{max})
\end{align}
$

In [4]:
#example
import torch
init_var = torch.rand((1,4))
trans_mat= torch.rand((4,4))
emit_score = torch.rand(4).view(-1,1)
print(init_var+trans_mat+emit_score)
#它是分别将init_var和_emit_score在相应维度复制m次

tensor([[2.6146, 2.7729, 2.1461, 2.2979],
        [1.5897, 1.2249, 1.3975, 1.7950],
        [1.7065, 1.4196, 1.4128, 1.8762],
        [1.9575, 2.2406, 1.4610, 1.4939]])


### Viterbi 解码算法  
计算最大路径得分，思想与HMM的解码算法类似，在序列t时刻计算和保存到第$Y_{j}^{t}$最大得分路径，最后回溯即可。阅读代码，初始前向最大得分变量，然后从Start Node节点转移到$y_{1}$,此时得分只有$[M_{transitions}+InitVar]$,由于初始节点除了start都为-10000，所以在t=1时刻保存的最大得分路径都是初始节点，通过torch.max返回得分矩阵在列上的最大索引，得到达到节点$y_{j}$的最大得分前一个节点，通过最大索引得到在t时刻各个节点的最大得分值。当从$y_{i}\rightarrow y_{i+1}$转移时，用最大的得分向量加上lstm输出打分 $feat_{i}$更新前向最大得分向量。当遍历所有序列到终止节点时，首先通过终止节点最大得分索引得到第T时刻节点，然后逐步回溯直至初始节点。

In [ ]:
# def viterbi_decode(self, feats):
#     backpointers = []  
#     # analogous to forward
#     init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
#     init_vvars[0][self.tag_to_ix[START_TAG]] = 0    # 前向最大得分变量，初始最大得分在Start节点。
#     forward_var = Variable(init_vvars)
#     if self.use_gpu:
#         forward_var = forward_var.cuda()
#     for feat in feats:
#         next_tag_var = forward_var.view(1, -1).expand(self.tagset_size, self.tagset_size) + self.transitions
#         _, bptrs_t = torch.max(next_tag_var, dim=1)  ##  索引最大得分的节点
#         bptrs_t = bptrs_t.squeeze().data.cpu().numpy()
#         next_tag_var = next_tag_var.data.cpu().numpy()
#         viterbivars_t = next_tag_var[range(len(bptrs_t)), bptrs_t]  #  索引节点最大得分
#         viterbivars_t = Variable(torch.FloatTensor(viterbivars_t))
#         if self.use_gpu:
#             viterbivars_t = viterbivars_t.cuda()
#         forward_var = viterbivars_t + feat  #  更新前向变量
#         backpointers.append(bptrs_t)

#     terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]  # 到达终止节点。
#     terminal_var.data[self.tag_to_ix[STOP_TAG]] = -10000.
#     terminal_var.data[self.tag_to_ix[START_TAG]] = -10000.
#     best_tag_id = argmax(terminal_var.unsqueeze(0))  # 到达终止节点最大得分的前置节点，即T时刻的节点。
#     path_score = terminal_var[best_tag_id]  # 最大得分
#     best_path = [best_tag_id]  # 保存 T节点，
#     for bptrs_t in reversed(backpointers):
#         best_tag_id = bptrs_t[best_tag_id]  # 向前索引
#         best_path.append(best_tag_id)
#     start = best_path.pop()
#     assert start == self.tag_to_ix[START_TAG]
#     best_path.reverse()  # 反转标注序列
#     return path_score, best_path